In [100]:
import zarr
import numpy as np
import pystac_client as pc
import xarray as xr
import rioxarray
import pandas as pd
from datetime import datetime, timezone, date
#from tqdm import tqdm
import BuildZarrStore as bzs
import stackstac
from pyproj import CRS
from dask.distributed import LocalCluster
#os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

In [101]:
origin = np.datetime64("2014-10-01")
date = np.datetime64(datetime.now()).astype("datetime64[D]")
date_delta = (date - origin+1).astype("int64")

date_delta

np.int64(3932)

In [102]:
x_extent = np.arange(4800000, 5700000, 20)
y_extent = np.arange(1800000, 1200000, -20)
time_extent = np.arange(0,date_delta,1)

In [103]:
shape = (time_extent.shape[0],y_extent.shape[0],x_extent.shape[0])
chunk_shape = (30,500,500)
shard_shape = (30,2500,2500)
x_shape = x_extent.shape
y_shape = y_extent.shape
time_shape = time_extent.shape

In [104]:
overwrite=True
store = zarr.storage.LocalStore("s1sig0_v2.zarr")
root = zarr.create_group(store=store, overwrite=overwrite)
vv = root.create_group("VV")
vh = root.create_group("VH")
meta = root.create_group("metadata")

In [105]:
bzs.make_group_S1(vv, "VV", shape, shards=shard_shape, chunks=chunk_shape, x_shape=x_shape, y_shape=y_shape, time_shape=time_shape, x_extent=x_extent, y_extent=y_extent, time_extent=time_extent)
bzs.make_group_S1(vh, "VH", shape, shards=shard_shape, chunks=chunk_shape, x_shape=x_shape, y_shape=y_shape, time_shape=time_shape, x_extent=x_extent, y_extent=y_extent, time_extent=time_extent)

In [106]:
meta.create_array(name="sensing_date",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int64",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"units": "seconds since 2014-10-01 00:00:00",
                            "calendar": "proleptic_gregorian",
                            "_FillValue": -9999},
                overwrite=overwrite)

meta.create_array(name="absolute_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int32",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999},
                overwrite=overwrite)

meta.create_array(name="relative_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999},
                overwrite=overwrite)

x_array = meta.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
            overwrite=overwrite)

y_array = meta.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
            overwrite=overwrite)

time_array = meta.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2014-10-01",
                        "calendar": "proleptic_gregorian"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

In [115]:
zarr.consolidate_metadata(store, path="VV")
zarr.consolidate_metadata(store, path="VH")
zarr.consolidate_metadata(store, path="metadata")
zarr.consolidate_metadata(store)

<Group file://s1sig0_v2.zarr>

In [108]:
store = zarr.storage.LocalStore("s1sig0_v2.zarr")
group = zarr.group(store=store)["VV"]
x_extent = group["x"][:]
y_extent = group["y"][:]
time_shape = group["time"].shape
data_shape = group["VV"].shape

In [109]:
time2append = np.arange(group["time"][-1]+1, group["time"][-1]+31, 1)

In [ ]:
group["VV"].resize((3962, 30000, 45000))

In [112]:
group["time"].append(time2append)

(3962,)

In [ ]:
with open('s1sig0_v2.zarr/zarr.json', 'w') as f:
    f.write("{}")

In [ ]:
zarr.consolidate_metadata(store, path="VV")
zarr.consolidate_metadata(store, path="VH")
zarr.consolidate_metadata(store, path="metadata")
zarr.consolidate_metadata(store)

In [116]:
xr.open_zarr("s1sig0_v2.zarr", group="VV", consolidated = True)

<xarray.Dataset> Size: 43TB
Dimensions:  (time: 3962, y: 30000, x: 45000)
Coordinates:
  * time     (time) datetime64[ns] 32kB 2014-10-01 2014-10-02 ... 2025-08-05
  * y        (y) float64 240kB 1.8e+06 1.8e+06 1.8e+06 ... 1.2e+06 1.2e+06
  * x        (x) float64 360kB 4.8e+06 4.8e+06 4.8e+06 ... 5.7e+06 5.7e+06
Data variables:
    VV       (time, y, x) float64 43TB dask.array<chunksize=(30, 500, 500), meta=np.ndarray>